In [7]:
import re
from src.model.synthesizer import InferenceConfig

from src.text.test_sentences import kur_vasara_amzina_sentences, witty_sentences
%matplotlib inline

## InoVoice TTS

In [8]:
device=0

configs = [
    # InferenceConfig(
    #     checkpoint_step=150000,
    #     hps=hps22,
    #     speaker='aurimas_nausedas',
    #     stressed=False,
    #     device=device,
    #     filelist_path=filelist_dir / 'vits_aurimas_ns.txt',
    #     output_dir=output_dir / 'vits_aurimas_ns'
    # ),
    # InferenceConfig(
    #     checkpoint_step=150000,
    #     hps=hps22,
    #     speaker='aurimas_nausedas',
    #     stressed=True,
    #     device=device,
    #     filelist_path=filelist_dir / 'filelist_stressed.txt',
    #     output_dir=output_dir / 'vits_aurimas_s'
    # ),
    # InferenceConfig(
    #     checkpoint_step=1305000,
    #     hps=hps22,
    #     speaker='giedrius_altoriu_sesely',
    #     stressed=False,
    #     device=device,
    #     filelist_path=filelist_dir / 'vits_giedrius_ns.txt',
    #     output_dir=output_dir / 'vits_giedrius_ns'
    # ),
    # InferenceConfig(
    #     checkpoint_step=1305000,
    #     hps=hps22,
    #     speaker='giedrius_altoriu_sesely',
    #     stressed=True,
    #     device=device,
    #     filelist_path=filelist_dir / 'vits_giedrius_s.txt',
    #     output_dir=output_dir / 'vits_giedrius_s'
    # ),
    # InferenceConfig(
    #     checkpoint_step=3020000,
    #     hps=hps22,
    #     speaker='vytautas',
    #     stressed=False,
    #     device=device,
    #     filelist_path=filelist_dir / 'vits_vytautas_ns.txt',
    #     output_dir=output_dir / 'vits_vytautas_ns'
    # ),
    # InferenceConfig(
    #     checkpoint_step=3020000,
    #     hps=hps22,
    #     speaker='vytautas',
    #     stressed=True,
    #     device=device,
    #     filelist_path=filelist_dir / 'vits_vytautas_s.txt',
    #     output_dir=output_dir / 'vits_vytautas_s'
    # ),
    # InferenceConfig(
    #     checkpoint_step=60000,
    #     config_name='aurimas_44khz',
    #     speaker='aurimas_nausedas_44',
    #     stressed=True,
    #     device=device,
    # ),
    # InferenceConfig(
    #     checkpoint_step=70000,
    #     config_name='aurimas_44khz',
    #     speaker='aurimas_44',
    #     stressed=True,
    #     device=device,
    # ),
    # InferenceConfig(
    #     checkpoint_step=55000,
    #     config_name='aurimas_studio_44khz',
    #     speaker='aurimas_studio_44',
    #     stressed=True,
    #     device=device,
    # ),
    # InferenceConfig(
    #     checkpoint_step=2175000,
    #     config_name='giedrius_44khz',
    #     speaker='giedrius_arbaciauskas_base',
    #     stressed=True,
    #     device=device,
    # ),
    # InferenceConfig(
    #     checkpoint_step=250000,
    #     config_name='giedrius_studio_44khz',
    #     speaker='giedrius_studio_44_ft_base_mel160',
    #     stressed=True,
    #     device=device,
    # ),
    # InferenceConfig(
    #     checkpoint_step=40000,
    #     config_name='milda_44khz',
    #     speaker='milda_44',
    #     stressed=True,
    #     device=device,
    # ),
    # InferenceConfig(
    #     checkpoint_step=87500,
    #     config_name='milda_no_noise_44khz',
    #     speaker='milda_no_noise_44',
    #     stressed=True,
    #     device=device,
    # ),
    # InferenceConfig(
    #     checkpoint_step=36000,
    #     config_name='arnas_44khz',
    #     speaker='arnas_44',
    #     stressed=True,
    #     device=device,
    # ),
    InferenceConfig(
        checkpoint_step=440000,
        config_name='ljs_base_44khz',
        speaker='ljs_base_44khz',
        stressed=False,
        device=device,
    ),
]

[2022-09-28 09:38:26,815] INFO:PID-954549:MainThread:src.model.checkpoint: Loading checkpoint at /home/aai-labs/inovoice/models/ljs_base_44khz/G_440000.pth
[2022-09-28 09:38:27,193] INFO:PID-954549:MainThread:src.model.checkpoint: Loaded checkpoint '/home/aai-labs/inovoice/models/ljs_base_44khz/G_440000.pth' (iteration 1613)


In [9]:
sents = [
    "- Jū̃s klaũsote sintezúoto tèksto.",
    "Vìlniaus universitèto Matemãtikos ir̃ informãtikos fakultètas.",
    "Vãsara paskutìnių kùrsų studeñtams kasmẽt prasìdeda įtemptaĩ - láukia baigiamų̃jų darbų̃ gynìmai. Vìlniaus universitèto Matemãtikos ir̃ informãtikos fakultetè šiaĩs mẽtais jiẽ prasìdeda gegužė̃s trisdešimt pirmąją ir̃ baĩgsis biržẽlio dešimtą diẽną.",
    "Žinaĩ, čià tóks ãtvejis - àš nemóku melúoti, todė̃l nesugebė́siu nutylė́ti, kàd tù turì trìs vaikùs.",
    "Blẽt naãchui pỹdaras negražìnsi skolõs - táu pyzdaã bùs!",
]
sents += witty_sentences + kur_vasara_amzina_sentences

In [10]:
english_sentences = [
    "paying particular attention to the crowd for any unusual activity.",
    "The only exit from the office in the direction Oswald was moving was through the door to the front stairway.",
    "the Commission is not able to reach any definite conclusions as to whether or not he was, quote, sane, unquote, under prevailing legal standards.",
    "After a few years of active exertion the Society was rewarded by fresh legislation.",
    "Three of these agents occupied positions on the running boards of the car, and the fourth was seated in the car.",
    "FBI, and the Secret Service.",
    "However, there is no evidence that these men failed to take any action in Dallas within their power that would have averted the tragedy.",
    "that he was the leader of a fascist organization, and when I said that even though all of that might be true, just the same he had no right to take his life,",
    "and stole paper on a much larger scale than Brown.",
    "A milliner's porter,",
    "After a moment's hesitation, she told me that he worked at the Texas School Book Depository near the downtown area of Dallas.",
]

In [11]:
sentences = [re.sub('[\u0300\u0301\u0303]', '', s) for s in english_sentences] if not configs[0].stressed else sents.copy()
# sentences = [re.sub('[\u0300\u0301\u0303]', '', s) for s in english_sentences] if not configs[0].stressed else sents.copy()

In [12]:
for config in configs:
    config.synthesizer.synthesize_all(sentences, plot_align=False, output_dir=config.output_dir)


[2022-09-28 09:38:31,600] INFO:PID-954549:MainThread:src.model.synthesizer: Synthesized 11 utterances.
